# Hint
- For Exercise 1, regex function should do (ensure that all edge cases are taken care of)

- For Exercise 2, build a corpus of products (built manually) and go from there. Mention the order of complexity (in any measures Big O, memory,
cpu etc.)

- For Exercise 2, our expectation is that you implement NER model

#Tech Stack

Your choice really. There are many out there NLTK, Spacy, Pytorch, TensorFlow etc.
#Duration/delivery
Revert back with github/gitlab/bitbucket link by pushing all of the code into the repo. And add clear readme.md setup instructions and any other
instructions within 3 days.

# Exercise 1
- Write a function that takes OfferDetails as input, and returns FaceValue as output

- Run the function against all rows in the attached data and determine function accuracy (Higher the better)

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import pandas as pd
import re
import numpy as np
import nltk
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree


df= pd.read_csv('/content/drive/My Drive/sks/coupons_ner.csv', names =['OfferDetails'])
df.head()

,OfferDetails
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...
2,Save $2.00 ONE Tide Laundry Detergent (exclude...
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...
4,$3.00 OFF when you purchase any THREE (3) Pepp...


# 1) Write a function that takes OfferDetails as input, and returns FaceValue as output

In [6]:
def extract_face_value(txt):
  p = re.compile(r'\$\d+\.[0-9]+|\$\.[0-9]+|\d+\¢|\$\d+') 
  x = p.search(txt)
  if x==None:
    return np.nan
  else:
    return x.group()

In [7]:
extract_face_value(str(df.loc[800]))

'$1.00'

# 2) Run the function against all rows in the attached data and determine function accuracy (Higher the better)

In [8]:
df['FaceValue']=df['OfferDetails'].apply(lambda x: extract_face_value(x))

In [9]:
df.head(7)

,OfferDetails,FaceValue
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...,$2.00
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...,$2.00
2,Save $2.00 ONE Tide Laundry Detergent (exclude...,$2.00
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...,$1.00
4,$3.00 OFF when you purchase any THREE (3) Pepp...,$3.00
5,SAVE $1.11 when you buy any ONE (1) Familly Si...,$1.11
6,SAVE $1.00 ON TWO when you buy TWO PACKAGES an...,$1.00


In [23]:
rows_with_nan = [index for index, row in df.iterrows() if row.isnull().any()]
print("Total number of missing FaceValue are:",len(rows_with_nan))

Total number of missing FaceValue are: 273


In [24]:
missing_facevalue_text={}
for i in rows_with_nan:
  missing_facevalue_text[i]=df['OfferDetails'].loc[i]

In [25]:
missing_facevalue_text

{131: "BUY ONE GET ONE FREE on any ONE (1) bag of Flavors of America by Eight O'Clock®, Any variety 11oz.",
 149: 'Golden Ripe Bananas PLU 4011 Per lb. Excludes Organic.',
 150: 'Signature SELECT refreshe Water 24-pack, 16.9-oz. +CRV.',
 151: 'Calidad Tortilla Chips 12 to 14-oz.',
 152: 'Chobani Greek Yogurt 5.3-oz.',
 153: 'Lucerne Shredded Cheese 6 to 8-oz.',
 154: 'Lucerne Butter 16-oz.',
 155: 'Mission Super Size Yellow Corn Tortillas 24-ct.',
 156: 'Barilla Pasta 12 to 16-oz. Excludes Plus Pasta and Gluten Free.',
 159: 'Signature Farms Romaine Hearts 3-ct. Limit 1.',
 160: 'Fresh Cravings Dip 8-10-oz. Limit 1.',
 161: 'Litehouse® Buy ONE (1) Litehouse® Organic Dressing(11.25 oz), get ONE...',
 162: 'Caspers Franks 16-oz. Limit 4.',
 163: 'Aidells Gourmet Chicken Sausage or Aidells meatballs. 12-oz. Limit 2 items.',
 164: 'Signature Farms 80% Lean Ground Beef Mega-pack. Limit 1.',
 165: 'Plated Meal Kits 22.2-32.5-oz.',
 166: 'Lipton or Tazo Tea 75-100-ct. Lipton or 20-ct. Tazo.',

**Observation**: 
1) After extracting FaceValues from OfferDetails feature, there are 273 rows where FaceValue is not given.

2) Using above function we are getting 100% correct result.


#  Exercise 2 : 1)Write a function that takes OfferDetails as input, and returns Product as output (Product can be single or array)

In [27]:
avoid=['OR','ONE','TWO','THREE','ON','BOXES','OZ','LARGE','SIZE','OFF','Size','SAVE','Save','Any','SMALL','Eight','TO','Includes','FREE','Max',\
  'Value','F','Only','GET','ONLY','700-ml','Limit','FOUR','SELECT','BUY','EXCLUDES','Studio','Buy']

def seperater(chunked):
  sub_trees=[]
  other=[]
  for i in chunked:
    if type(i) == Tree:
      t=" ".join([token for token, pos in i.leaves() if token not in avoid])
      if len(t.split())>=1:
        sub_trees.append(t)
    else :
      other.append(i)
  return sub_trees,other


def product_names(text):
  out=[]
  collect=[]

  chunked= nltk.ne_chunk(pos_tag(word_tokenize(text)), binary=True)
  out,remain=seperater(chunked)   
  
  for i,data in enumerate(remain):
    # current one and next one are real NNP's , next element is not last one 
    # NNP---NNP---NNP........
    if (data[1]=="NNP" and data[0] not in avoid) and i<=len(remain)-2 and remain[i+1][1]=='NNP' :
      collect.append(data[0])
    # not an NNP word
    elif data[1]=="NNP" and data[0] in avoid:
      t=" ".join(collect)
      if len(t.split())>=1:
        out.append(t)
        collect=[]
    # NNP - JS
    elif data[1]=="NNP" and i<=(len(remain)-2) and remain[i+1][1]!='NNP' and data[0] not in avoid:
      collect.append(data[0])
      t=" ".join(collect)
      if len(t.split())>=1:
        out.append(t)
        collect=[]
    # last real NNP
    elif (i)==(len(remain)-1) and data[1]=="NNP" and data[0] not in avoid :
      collect.append(data[0])
      t=" ".join(collect)
      if len(t.split())>=1:
        out.append(t)
        collect=[]
    
  return out

In [36]:
input = str(df['OfferDetails'].loc[2])
input

'Save $2.00 ONE Tide Laundry Detergent (excludes Tide Purclean Laundry Detergent, Studio by Tide Laundry Detergent, Tide PODS, Tide Rescue, Tide Simply Laundry Detergent, Tide Simply PODS, Tide Detergent 10 oz and trial/travel size).'

In [37]:
products = product_names(input)
print("Product Names: \n")
for i in products:
  print(i)

Product Names: 

Tide Laundry Detergent
Tide PODS
Tide Rescue
Tide Simply Laundry Detergent
Tide Simply PODS
Tide
Tide Laundry Detergent
Tide Purclean Laundry Detergent
Detergent


### **Observation** :

Above function is able to retrive product names with more than 90% accurate result. 

# 2) Run the function against all rows in the attached data and determine function accuracy (Higher the better)

In [30]:
df['Product'] = df['OfferDetails'].apply(lambda x: product_names(x))

In [31]:
df.head(3)

,OfferDetails,FaceValue,Product
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...,$2.00,"[Downy Odor, Wash Scent, Downy Unstopables, Fr..."
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...,$2.00,"[Tide Simply, Tide Simply PODS, Tide, Tide POD..."
2,Save $2.00 ONE Tide Laundry Detergent (exclude...,$2.00,"[Tide Laundry Detergent, Tide PODS, Tide Rescu..."


In [38]:
df.to_csv('/content/drive/My Drive/sks/output.csv')

###  For Exercise 2, Mention the order of complexity (in any measures Big O, memory,cpu etc.)

# Order of complexity : O(n)